In [1]:
import pickle
import pandas as pd
import numpy as np
from itertools import groupby

from zlu_functions import *
from nel_functions import *
from helper import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
problematic0 = pd.DataFrame(
    {
    'damn' : ['date', ''     , ''     , ''     , 2, 3, ''     , ''     , 5, 5, ''     , 33],
    'damn1': [1     , 'shitA', 'shitD', 'shitG', 2, 4, 'shitG', 'shitG', 3, 6, 'shitG', 33],
    'damn2': [1     , 'shitB', 'shitE', 'shitH', 2, 3, 'shitH', 'shitH', 2, 7, 'shitG', 33],
    'damn3': [1     , 'shitC', 'shitF', 'shitI', 2, 3, 'shitI', 'shitI', 3, 8, 'shitG', 33]
    }
)

In [3]:
problematic0 = problematic0.applymap(str)
problematic0

,damn,damn1,damn2,damn3
0,date,1,1,1
1,,shitA,shitB,shitC
2,,shitD,shitE,shitF
3,,shitG,shitH,shitI
4,2,2,2,2
5,3,4,3,3
6,,shitG,shitH,shitI
7,,shitG,shitH,shitI
8,5,3,2,3
9,5,6,7,8


In [4]:
df_original = problematic0.copy()
x_idx = []
if isinstance(df_original, pd.DataFrame) & (not df_original.empty):        
    for x in range(df_original.shape[0]):
        boolean_row = []
        for y in range(df_original.shape[1]):
            if df_original.iloc[x, y] != '':
                df_original.iloc[x, y] = re.sub(r'\([^()]*\)', '', df_original.iloc[x, y])
                if re.findall('[\d]', df_original.iloc[x, y]) == []:
                    boolean_row.append(True)
                else: boolean_row.append(False)
        if np.array(boolean_row).all():
            x_idx.append(x)

In [5]:
df_list = []
x_idx_np = np.array(x_idx)
mask = np.diff(x_idx_np) != 1
mask2 = np.diff(x_idx_np) == 1
grouper = [list(group) for k, group in groupby(mask2, lambda x: x == False) if not k]
subtractor = np.array([len(x) for x in grouper])
x_idx_np = np.append(x_idx_np[0], x_idx_np[1:][mask])
x_idx_loop = np.append(x_idx_np[0], np.diff(x_idx_np) - 1 - subtractor)
loop = len(x_idx_loop)
for index in x_idx_loop:
    df_split1 = df_original.iloc[:index]
    df_split2 = df_original.iloc[index:]
    columns_temp = df_split2.columns
    df_split2.columns = ["" for x in columns_temp]
    #reset col headers; if empty using the col headers of the parent df
    df_split2 = set_col_headers(df_split2) 
    df_split2.columns = [columns_temp[k] if re.search('[\w]',v)==None else v for k,v in enumerate(df_split2.columns)]
    df_list.append(df_split1)
    df_original = df_split2
    loop -=1
    if loop == 0:
        df_list.append(df_split2)

In [6]:
len(df_list)

4

In [7]:
df_list[0]
df_list[1]
df_list[2]
df_list[3]

,damn,damn1,damn2,damn3
0,date,1,1,1


,damn,"shitA,shitD,shitG","shitB,shitE,shitH","shitC,shitF,shitI"
4,2,2,2,2
5,3,4,3,3


,damn,"shitG,shitG","shitH,shitH","shitI,shitI"
8,5,3,2,3
9,5,6,7,8


,damn,shitG,shitG,shitG
11,33,33,33,33
